In [1]:
!pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

  Using cached labelme-5.1.1.tar.gz (1.5 MB)
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for labelme: filename=labelme-5.1.1-py3-none-any.whl size=1466357 sha256=57cdca92d4c066233c838b84b36dbf59dfbc3b83491bcbad51c9b76cd6ebf56c
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\f7\dd\4b\aa34a2cc4e9a53f43b682ee3e74ab03e877f37cc36c67ea099
  Running setup.py clean for tensorflow-gpu
  Created wheel for imgviz: filename=imgviz-1.7.0-py3-none-any.whl size=7675291 sha256=e36615712dc6a72adaa82f24034992e1448a9cd34092fcdf39e50929064d36f4
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\9d\b2\0f\9d88f7bcd14fe2c3cf7f6b271aa01ce2af8

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\asus\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\asus\\AppData\\Local\\Temp\\pip-install-aglyrjjy\\tensorflow-gpu_c18c0ebf0e8b459c8f9d7976aba82ef3\\setup.py'"'"'; __file__='"'"'C:\\Users\\asus\\AppData\\Local\\Temp\\pip-install-aglyrjjy\\tensorflow-gpu_c18c0ebf0e8b459c8f9d7976aba82ef3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\asus\AppData\Local\Temp\pip-wheel-7d_tntrf'
       cwd: C:\Users\asus\AppData\Local\Temp\pip-install-aglyrjjy\tensorflow-gpu_c18c0ebf0e8b459c8f9d7976aba82ef3\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\a

In [ ]:
import 